In [1]:
import numpy as np
from skimage.color import rgb2gray
from skimage.feature import graycomatrix, graycoprops
import tensorflow as tf
from tensorflow.keras import layers, Model

In [2]:
class GLCMModule(tf.keras.layers.Layer):
    def __init__(self, distances=[1]):
        super().__init__()
        self.distances = distances
        self.angles = [0, np.pi/8, np.pi/4, 3*np.pi/8,
                       np.pi/2, 5*np.pi/8, 3*np.pi/4, 7*np.pi/8]

    def call(self, x):
        out = tf.py_function(self._glcm_numpy, [x], Tout=tf.float32)
        out.set_shape((None, 24))  
        return out

    def _glcm_numpy(self, x_np):
        x_np = np.array(x_np)
        batch_features = []
        for sample in x_np:
            feats = []
            for d in range(sample.shape[2]):  # D
                frame = sample[:, :, d, :]
                if frame.shape[-1] == 1:
                    gray = frame[..., 0]
                else:
                    gray = rgb2gray(frame)
                gray = (gray * 255).astype(np.uint8)

                glcm = graycomatrix(gray, distances=self.distances,
                                    angles=self.angles, symmetric=True, normed=True)
                contrast = graycoprops(glcm, 'contrast').mean()
                dissimilarity = graycoprops(glcm, 'dissimilarity').mean()
                homogeneity = graycoprops(glcm, 'homogeneity').mean()
                ASM = graycoprops(glcm, 'ASM').mean()
                energy = np.sqrt(ASM)
                std = np.std(gray)
                feats.extend([std, contrast, dissimilarity, homogeneity, ASM, energy])
            batch_features.append(feats)
        return np.array(batch_features, dtype=np.float32)

In [3]:
class SpatioTemporalAttention(layers.Layer):
    """Simple 3D attention: mean over channels → conv3d(1) → sigmoid → multiply"""
    def __init__(self, kernel_size=(3,3,3)):
        super().__init__()
        self.conv = layers.Conv3D(1, kernel_size, padding='same', use_bias=True)
        self.act = layers.Activation('sigmoid')

    def call(self, x):
        att = tf.reduce_mean(x, axis=-1, keepdims=True)
        att = self.conv(att)
        att = self.act(att)
        return x * att

In [4]:
def build_enhanced_3d_cnn(input_shape=(128, 128, 4, 1), feature_dim=4, name='enhanced_3dcnn'):
    """
    Enhanced 3D CNN architecture with spatiotemporal attention.
    Output feature vector size = 4 .
    """
    inp = layers.Input(shape=input_shape)

    # 🔹 Step 1: Spatiotemporal Attention
    x = SpatioTemporalAttention()(inp)

    # 🔹 Block 1
    x = layers.Conv3D(8, (3, 3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool3D(pool_size=(2, 2, 1), padding='same')(x)

    # 🔹 Block 2
    x = layers.Conv3D(16, (3, 3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)

    # 🔹 Block 3
    x = layers.Conv3D(32, (3, 3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)

    # 🔹 Block 4
    x = layers.Conv3D(64, (3, 3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)

    # 🔹 Block 5
    x = layers.Conv3D(128, (3, 3, 3), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPool3D(pool_size=(2, 2, 2), padding='same')(x)

    # 🔹 Feature extraction head
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(feature_dim, use_bias=False)(x)   
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    return tf.keras.Model(inp, x, name=name)

In [5]:
from skimage.feature import local_binary_pattern
    """LBP module that forwarded to the 3d cnn"""
class LBPCNN(tf.keras.Model):
    def __init__(self, enhanced_cnn):
        super().__init__()
        self.enhanced_cnn = enhanced_cnn

    def call(self, face_3d):
        lbp_vols = tf.py_function(self._lbp_numpy, [face_3d], Tout=tf.float32)
        lbp_vols.set_shape((None, 128, 128, 4, 1))  
        return self.enhanced_cnn(lbp_vols)

    def _lbp_numpy(self, x_np):
        x_np = np.array(x_np)
        lbp_vols = []
        for sample in x_np:
            H, W, D, C = sample.shape
            lbp_stack = np.zeros((H, W, D, 1), dtype=np.float32)
            for d in range(D):
                frame = sample[:, :, d, :]
                if frame.shape[-1] == 1:
                    gray = frame[..., 0]
                else:
                    gray = rgb2gray(frame)
                lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
                lbp_stack[:, :, d, 0] = lbp / (lbp.max() + 1e-6)
            lbp_vols.append(lbp_stack)
        return np.array(lbp_vols, dtype=np.float32)

In [6]:
class SiameseNetwork(Model):
    """Shared Enhanced3DCNN for face and background"""
    def __init__(self, shared_cnn):
        super().__init__()
        self.shared_cnn = shared_cnn

    def call(self, inputs):
        face_3d, back_3d = inputs
        f_feat = self.shared_cnn(face_3d)
        b_feat = self.shared_cnn(back_3d)
        return f_feat, b_feat

In [7]:
class SLPClassifier(Model):
    """Single dense unit with sigmoid"""
    def __init__(self):
        super().__init__()
        self.fc = layers.Dense(1, activation='sigmoid')

    def call(self, x):
        return self.fc(x)

In [8]:
class DeepfakeDetectionModel(tf.keras.Model):
    def __init__(self, glcm_module, lbp_cnn, siamese_net, slp_classifier):
        super().__init__()
        self.glcm = glcm_module
        self.lbp_cnn = lbp_cnn
        self.siamese = siamese_net
        self.slp = slp_classifier

    def call(self, face_3d, back_3d):
      
        glcm_feats = self.glcm(face_3d)
        lbp_feats = self.lbp_cnn(face_3d)
        face_feats, back_feats = self.siamese([face_3d, back_3d])
        fused = tf.concat([glcm_feats, lbp_feats, face_feats, back_feats], axis=-1)
        return self.slp(fused)


In [ ]:
glcm_module = GLCMModule()
enhanced_cnn = build_enhanced_3d_cnn(input_shape=(128,128,4,1), feature_dim=4)
lbp_cnn = LBPCNN(enhanced_cnn)
siamese_net = SiameseNetwork(enhanced_cnn)
slp_classifier = SLPClassifier()

model = DeepfakeDetectionModel(glcm_module, lbp_cnn, siamese_net, slp_classifier)